In [1]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

env = make_vec_env(
    "HalfCheetah-v4",
    rng=np.random.default_rng(),
    n_envs=4,
    env_make_kwargs={"render_mode": "rgb_array"},
)


expert = load_policy(
    "sac-huggingface",
    organization="sb3",
    env_name="HalfCheetah-v3",
    venv=env,
)

obs = env.reset()
print("num_env", env.num_envs)
print("num observation", env.observation_space.shape[0])
print("num action", env.action_space.shape[0])

C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: 'bytes' object cannot be interpreted as an integer
  warnings.warn(
C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


num_env 4
num observation 17
num action 6


In [2]:
from src.models.dagger import TrajectoryAccumulator
from src.models.dagger import generate_trajectories

In [3]:
from src.models.dagger import ReplayBuffer
from src.models.dagger import BC
from src.models.dagger import SimpleDAggerTrainer

In [4]:
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [5]:
bc_trainer = BC(
    observation_space=env.observation_space, 
    action_space=env.action_space, 
    rng=np.random.default_rng())

observation_space Box(-inf, inf, (17,), float64)


In [6]:
dagger_trainer = SimpleDAggerTrainer(
    venv=env,
    expert_policy=expert,
    bc_trainer=bc_trainer,
    rng=np.random.default_rng(),
)

In [ ]:
dagger_trainer.train(1)

round:  0
partial_trajectories {0: [], 1: [], 2: [], 3: []}
Start generate Traj
finish_trajectory
finish_trajectory
finish_trajectory
finish_trajectory
active env sum  0
active env [False False False False]
END


Training :   0%|                                                                               | 0/125 [00:00<?, ?it/s]